In [ ]:
# imports
import sys

# include path to bubble finding algorithm here
sys.path.insert(1, '/mnt/home/wallinbr/Python3.6.4/BubbleFindingAlgorithm/.')

import warnings
warnings.filterwarnings("ignore")

from bubble_algorithm_class import *
from bubble_plotting_functions import *
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import yt
from unyt import Gyr

# libraries used only for this file
from numba import jit
import treecorr

In [ ]:
################################################################################################
# Variables to change based on galaxy images used
################################################################################################
'''Variables:
    - seed: random seed for numpy to keep noise algorithm results the same (optional)
    - smooth_ims: boolean to decide whether to apply a smoothing gaussian to images based
                  on the observation galaxy's beam size
    - smooth: multiple of observation image's beam used to create smoothing gaussian
    - pixel_size: angular pixel side length for observation galaxy image (arcsec)
    - beam_size: beam semimajor and semiminor axis from observation galaxy image (arcsec)
    - obs_gal_dist: physical distance to observation galaxy (kpc)
    - sim_diam_kpc: physical size of simulationgalaxy image if used (kpc)
    - obs_file: file path of saved observation column density data .npy file (exlude extension)
    - obs_file: file path of saved simulation column density data .npy file (exlude extension)
    '''

seed = 92947490
smooth_ims = True
smooth = 2
pixel_size = 1.5   # arecseconds
beam_size = np.array([6.04, 6.04])   # arcseconds
obs_gal_dist = 5.9*1000   # kpc
sim_diam_kpc = 40   # kpc

obs_file = "NGC6946"
sim_file = "inclined_atomicH_colden"

################################################################################################
# Loading in galaxy images
################################################################################################
obs_im = np.load(obs_file + ".npy")   # H*cm^-2
sim_im = np.load(sim_file + ".npy")   # H*cm^-2

obs_im = np.flip(obs_im, axis = 0)

################################################################################################
# Finding physical size of each real galaxy
################################################################################################

# angular to physical size formula for some real galaxy
obs_pixel_size = pixel_size / 206265  # arcsec -> rad
obs_diam_kpc = (obs_pixel_size*obs_im.shape[0]) * obs_gal_dist   # kpc

################################################################################################
# Applying the observation image's beam and noise to the simulation image
################################################################################################

sim_beam = apply_beam(beam_size, sim_im.shape[0], pixel_size, sim_im)   # sim image with beam
sim_noise_beam, _ = sim_noise_alg(sim_beam, obs_im, seed)   # sim image with noise and beam

################################################################################################
# Using the Apply Beam algorithm to smooth simulation and observation images (optional)
################################################################################################

if(smooth_ims == True):
    # smoothing each galaxy with a gaussian of size a multiple of observation's beam size
    smoothing_axes = smooth*beam_size
    out_obs_im = apply_beam(smoothing_axes, obs_im.shape[0], pixel_size, obs_im)
    out_sim_im = apply_beam(smoothing_axes, sim_noise_beam.shape[0], pixel_size, sim_noise_beam)

else:
    out_obs_im = obs_im
    out_sim_im = sim_noise_beam

## Star Mass in Bubbles

In [ ]:
'''Functions for finding distributions'''

def histogram_sample(data_x, data_y, num, max_val):
    data_rad = np.sqrt(data_x**2 + data_y**2)
    min_val = np.min(data_rad)
    rad_arr = np.geomspace(min_val, max_val, num)
    n, b, _ = plt.hist(data_rad, bins = rad_arr)
    plt.close()
    bin_endpoints = rad_arr[1:]
    cdf = np.cumsum(n)
    cdf /= cdf[-1]

    return cdf, bin_endpoints

def random_sample(cdf, bins, N):
    values = np.random.rand(N)
    value_bins = np.searchsorted(cdf, values)
    random_from_cdf = bins[value_bins]

    return random_from_cdf

### Spatial Corss-Correlation of Star Particles and Superbubbles

In [ ]:
#####################################################
# Collecting all pre-processing items together
#####################################################


'''Functions for finding distance between each point and the number of pairs'''

@jit
def data_pairs(data1, data2):
    # creating matrix for all distance pairs
    dist_matrix = np.zeros(shape = (len(data1), len(data2)))
    
    # finding all distances between data points
    for i in range(len(data1)):
        for j in range(len(data2)):
            dist_matrix[i, j] = np.sqrt((data1[i,0]-data2[j,0])**2 + (data1[i,1]-data2[j,1])**2)
    
    return dist_matrix

def find_pairs(dist_matrix, r_0, r):
    # finding distances within radius
    upper_dist = dist_matrix <= r
    lower_dist = dist_matrix > r_0
    comb_mask = np.logical_and(upper_dist, lower_dist)
    
    # counting number of pairs
    data_keep = dist_matrix[comb_mask]
    pairs = data_keep.shape[0]
    
    return pairs


'''Trying to find spatial cross correlation of star particles and superbubbles'''

star_x = np.load("inclined_x.npy")
star_y = np.load("inclined_y.npy")


'''Finding uninclined star particle locations to construct distribution to then incline'''

o_star_coords = np.load("original_coords.npy")


'''Collecting star location and bubble centroids'''

# running superbubble finding algorithm
galaxy_obj = Bubble_Alg(out_sim_im, sim_diam_kpc, 50, beam_size[0], pixel_size)

# finding all cell locations where star mass exists
star_loc = []
for loc in zip(star_x, star_y): 
    star_loc.append(loc)
star_loc = np.array(star_loc)
print("Num stars:", len(star_loc))

# finding superbubble center locations
bub_centroids_x = []
bub_centroids_y = []

for num in galaxy_obj.bubble_arr:
    convert_arr = (np.array(galaxy_obj.info_dict[num-1]["centroid"])-512)*(40/1024)
    bub_centroids_x.append(convert_arr[1])
    bub_centroids_y.append(-1*convert_arr[0])

bub_centroids = [i for i in zip(bub_centroids_x, bub_centroids_y)]
bub_centroids = np.array(bub_centroids)
print("Num superbubbles:", len(bub_centroids))


'''Getting set of possible points for random point selection'''

galaxy_loc = []
not_nan = np.logical_not(np.isnan(galaxy_obj.ext_rem_im))
galaxy_where = np.where(not_nan)

for loc in zip(galaxy_where[1], galaxy_where[0]): 
    galaxy_loc.append(loc)
    
galaxy_loc = (np.array(galaxy_loc)-512)*(40/1024)   # kpc
galaxy_loc[:, 1] *= -1


'''finding distribution of stars and bubbles'''

theta = np.radians(33)
uninclined_bub_centroids = np.copy(bub_centroids)
uninclined_bub_centroids[:,1] /= np.cos(theta)

# finding distribution of uninclined star particles and superbubbles
num_bins = 1000
bub_cdf, bub_bins = histogram_sample(uninclined_bub_centroids[:,0], uninclined_bub_centroids[:,1], num_bins, 20)
star_cdf, star_bins = histogram_sample(o_star_coords[:,0], o_star_coords[:,1], num_bins, 20)

In [ ]:
'''Masking data points into quadrants'''

def create_masks(arr):
    pos_x_mask = np.ma.getmaskarray(np.ma.masked_greater_equal(arr[:, 0], 0))
    pos_y_mask = np.ma.getmaskarray(np.ma.masked_greater_equal(arr[:, 1], 0))
    neg_x_mask = np.ma.getmaskarray(np.ma.masked_less(arr[:, 0], 0))
    neg_y_mask = np.ma.getmaskarray(np.ma.masked_less(arr[:, 1], 0))

    pp_mask = np.logical_and(pos_x_mask, pos_y_mask)
    pn_mask = np.logical_and(pos_x_mask, neg_y_mask)
    np_mask = np.logical_and(neg_x_mask, pos_y_mask)
    nn_mask = np.logical_and(neg_x_mask, neg_y_mask)
    
    return [pp_mask, np_mask, nn_mask, pn_mask]
    

# stars
m = create_masks(star_loc)

Q1_stars = star_loc[m[0]]
Q2_stars = star_loc[m[1]]
Q3_stars = star_loc[m[2]]
Q4_stars = star_loc[m[3]]

del(star_loc)


# galaxy locations
m = create_masks(galaxy_loc)

Q1_galaxy = galaxy_loc[m[0]]
Q2_galaxy = galaxy_loc[m[1]]
Q3_galaxy = galaxy_loc[m[2]]
Q4_galaxy = galaxy_loc[m[3]]

del(galaxy_loc)


# superbubble locations
m = create_masks(bub_centroids)

Q1_bubs = bub_centroids[m[0]]
Q2_bubs = bub_centroids[m[1]]
Q3_bubs = bub_centroids[m[2]]
Q4_bubs = bub_centroids[m[3]]

del(bub_centroids)

In [ ]:
'''Running rest of correlation code by quadrant'''

xi_dict = {}
xi_tc_dict = {}
var_tc_dict = {}
pair_dict = {}

galaxy = [Q1_galaxy, Q2_galaxy, Q3_galaxy, Q4_galaxy]
stars = [Q1_stars, Q2_stars, Q3_stars, Q4_stars]
bubs = [Q1_bubs, Q2_bubs, Q3_bubs, Q4_bubs]

D1_patches = {}
D2_patches = {}
R1_patches = {}
R2_patches = {}

count = 0
for j in range(len(galaxy)):
    # creating galaxy out of 3 quadrants only
    quad_gal = galaxy.pop(j)
    quad_stars = stars.pop(j)
    quad_bubs = bubs.pop(j)
    
    use_gal = np.concatenate(galaxy, axis = 0)
    use_stars = np.concatenate(stars, axis = 0)
    use_bubs = np.concatenate(bubs, axis = 0)

    
    '''New random point code following distribution'''

    # number of random points to use
    N1 = 10*len(use_stars)
    N2 = 10*len(use_bubs)

    # finding random angles to use
    if(j == 0):
        R1_ang = np.random.uniform(np.pi/2, 2*np.pi, N1)
        R2_ang = np.random.uniform(np.pi/2, 2*np.pi, N2)
    elif(j == 1):
        R1_ang = np.random.uniform(np.pi, 5*np.pi/2, N1)
        R2_ang = np.random.uniform(np.pi, 5*np.pi/2, N2)
    elif(j == 2):
        R1_ang = np.random.uniform(3*np.pi/2, 3*np.pi, N1)
        R2_ang = np.random.uniform(3*np.pi/2, 3*np.pi, N2)
    elif(j == 3):
        R1_ang = np.random.uniform(0, 3*np.pi/2, N1)
        R2_ang = np.random.uniform(0, 3*np.pi/2, N2)

    # randomly sampling star and bubble radii from actual distribution
    R1_rad = random_sample(star_cdf, star_bins, N1)
    R2_rad = random_sample(bub_cdf, bub_bins, N2)

    # initializing cartesian array
    R1 = np.zeros(shape = (N1, 2))
    R2 = np.zeros(shape = (N2, 2))

    # angle for inclining y-coordinate
    inc_angle = np.radians(33)

    # converting polar to cartesian - inclining to match galaxy
    R1[:, 0] = R1_rad*np.cos(R1_ang)
    R1[:, 1] = R1_rad*np.sin(R1_ang)*np.cos(inc_angle)
    R2[:, 0] = R2_rad*np.cos(R2_ang)
    R2[:, 1] = R2_rad*np.sin(R2_ang)*np.cos(inc_angle)
    
    # saving patches for TreeCorr code below
    D1_patches[j] = use_stars
    D2_patches[j] = use_bubs
    R1_patches[j] = R1
    R2_patches[j] = R2

    # finding the distance between sets of points
    print("D1D2")
    D1D2_dist = data_pairs(use_stars, use_bubs)
    print("\nD1R2")
    D1R2_dist = data_pairs(use_stars, R2)
    print("\nR1D2")
    R1D2_dist = data_pairs(R1, use_bubs)
    print("\nR1R2")
    R1R2_dist = data_pairs(R1, R2)
    print("\nDistance Calculation Complete\n")
    
    
    # checking minimum distances
    print("star bubble min distance (kpc):", np.min(D1D2_dist))
    print("star random 2 min distance (kpc):", np.min(D1R2_dist))
    print("random 1 bubble min distance (kpc):", np.min(R1D2_dist))
    print("random 1 random 2 min distance (kpc):", np.min(R1R2_dist), "\n")
    
    
    '''Finding spatial cross-correlation'''

    # radial distances to mask over
    r_use = np.geomspace(0.1, 5, 20, endpoint = False)

    # normalization factors
    N1_norm = len(use_stars)/len(R1)
    N2_norm = len(use_bubs)/len(R2)

    print("Norm1", N1_norm)
    print("Norm2", N2_norm)

    pair_dict["D1D2"] = []
    pair_dict["D1R2"] = []
    pair_dict["R1D2"] = []
    pair_dict["R1R2"] = []

    # calculating spatial cross-correlation given each radius
    xi_arr = np.zeros(len(r_use)-1, dtype = np.float64)
    for i in range(len(r_use)-1):
        r_0 = r_use[i]
        r = r_use[i+1]

        # finding total number of pairs
        D1D2 = find_pairs(D1D2_dist, r_0, r)
        D1R2 = find_pairs(D1R2_dist, r_0, r)
        R1D2 = find_pairs(R1D2_dist, r_0, r)
        R1R2 = find_pairs(R1R2_dist, r_0, r)

        pair_dict["D1D2"].append(D1D2)
        pair_dict["D1R2"].append(D1R2)
        pair_dict["R1D2"].append(R1D2)
        pair_dict["R1R2"].append(R1R2)

        # calculating spatial cross-correlation coefficient
        try:
            xi = (D1D2 - D1R2*N2_norm - R1D2*N1_norm + R1R2*N1_norm*N2_norm)/(R1R2*N1_norm*N2_norm)
            xi_arr[i] = xi
        except:
            xi_arr[i] = np.nan
            pass

    xi_dict[count] = xi_arr
    count += 1
    
    galaxy.insert(j, quad_gal)
    stars.insert(j, quad_stars)
    bubs.insert(j, quad_bubs)

In [ ]:
'''TreeCorr Correlation Code'''

# initializing count-count correlation solver
nn = treecorr.NNCorrelation(nbins = 20, min_sep = 1e-1, max_sep = 5, var_method='jackknife')
rr = treecorr.NNCorrelation(nbins = 20, min_sep = 1e-1, max_sep = 5, var_method='jackknife')
dr = treecorr.NNCorrelation(nbins = 20, min_sep = 1e-1, max_sep = 5, var_method='jackknife')
rd = treecorr.NNCorrelation(nbins = 20, min_sep = 1e-1, max_sep = 5, var_method='jackknife')

# empty lists to append
D1_all = []
D2_all = []
R1_all = []
R2_all = []

D1_patch_num = []
D2_patch_num = []
R1_patch_num = []
R2_patch_num = []

# running through each patch
for i in D1_patches.keys():
    D1_all.append(D1_patches[i])
    D2_all.append(D2_patches[i])
    R1_all.append(R1_patches[i])
    R2_all.append(R2_patches[i])

    D1_patch_num.append(i*np.ones(len(D1_patches[i]), dtype = np.int64))
    D2_patch_num.append(i*np.ones(len(D2_patches[i]), dtype = np.int64))
    R1_patch_num.append(i*np.ones(len(R1_patches[i]), dtype = np.int64))
    R2_patch_num.append(i*np.ones(len(R2_patches[i]), dtype = np.int64))

# combining all data
D1_all = np.concatenate(D1_all)
D2_all = np.concatenate(D2_all)
R1_all = np.concatenate(R1_all)
R2_all = np.concatenate(R2_all)

# assigning patch number to each element in all data array
D1_patch_num = np.concatenate(D1_patch_num)
D2_patch_num = np.concatenate(D2_patch_num)
R1_patch_num = np.concatenate(R1_patch_num)
R2_patch_num = np.concatenate(R2_patch_num)

# creating catalog using patches
star_cat = treecorr.Catalog(x=D1_all[:,0], y=D1_all[:,1], patch = D1_patch_num)
bub_cat = treecorr.Catalog(x=D2_all[:,0], y=D2_all[:,1], patch = D2_patch_num)
R1_cat = treecorr.Catalog(x=R1_all[:,0], y=R1_all[:,1], patch = R1_patch_num)
R2_cat = treecorr.Catalog(x=R2_all[:,0], y=R2_all[:,1], patch = R2_patch_num)

# processing cross correlation
nn.process(star_cat, bub_cat)
rr.process(R1_cat, R2_cat)
dr.process(star_cat, R2_cat)
rd.process(R1_cat, bub_cat)

# calculating cross correlation values
xi_tc, varxi_tc = nn.calculateXi(rr = rr, dr = dr, rd = rd)

In [ ]:
'''Using jacknife estimator for statistical analysis and error bars'''

# getting each jackknife combination correlation values
n = len(xi_dict.keys())

# finding jackknife mean
jackknife_mean = np.zeros_like(xi_dict[0], dtype = np.float64)
for j in xi_dict.keys():
    jackknife_mean += xi_dict[j]

jackknife_mean /= n

# finding jackknife standard error
jackknife_var = np.zeros(shape = (len(xi_dict[0]), len(xi_dict[0])), dtype = np.float64)
for k in xi_dict.keys():
    v = xi_dict[k] - jackknife_mean
    v_shape = len(v)
    v = v.reshape(1, v_shape)
    v_t = v.reshape(v_shape, 1)
    jackknife_var += np.matmul(v_t, v)
    
jackknife_var *= (n-1)/n

jackknife_error_bars = np.zeros_like(xi_dict[0], dtype = np.float64)
for i in range(len(xi_dict[0])):
    jackknife_error_bars[i] = np.sqrt(jackknife_var[i,i])

log_r = np.log(r_use)
mid = (np.log(r_use[1])-np.log(r_use[0]))/2
mid_bins = np.exp(log_r+mid)
mid_bins = mid_bins[:-1]

ew = 0.2
plt.errorbar(mid_bins, jackknife_mean, yerr = jackknife_error_bars, color = "blue", 
             capsize = 0.5, elinewidth = ew, label = " Our Code")

plt.errorbar(np.exp(nn.logr), xi_tc, yerr = varxi_tc, color = "Red", 
             capsize = 0.5, elinewidth = ew, label = " TC")

plt.ylabel("Spatial Cross-Correlation")
plt.xlabel(r"Radial Separation $\left(\mathrm{kpc}\right)$")
plt.legend()

plt.xscale("log")
plt.ylim([-1,None])